In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
!pip install surprise
from surprise import Reader, Dataset
from surprise import SVD, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, SVDpp
from surprise.model_selection import GridSearchCV
from surprise import accuracy

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619419 sha256=fc1b040834a8fd5516abb8571a59dc0859557a317f108bbc4c6ceab6523e1b11
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
data_read.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(853486, 19)

In [ ]:
df = data_read.sample(frac=0.10, random_state=42)

In [ ]:
df = df.rename(columns={'username': 'userID','Name':'itemID','user_overall_score':'rating'})
df_copy = df.copy()

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import re
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import linear_kernel
import random
import pickle

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
df = data_read.copy()
df = df.sample(frac=0.10, random_state=1)

# Remove Punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
df['text'] = df['text'].apply(remove_punctuations)

# Make Strings Lowercase
df['text'] = df['text'].str.lower()

# Remove Digits
df['text'] = df['text'].str.replace('\d+', '')

#define lemmatizer
lemmatizer = WordNetLemmatizer()

#remove stopwords
stop = stopwords.words('english')
df['without_stopwords']  = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#lemmatize/tokenize/make everything lowercase/remove punctuation
df['lemmatized_text'] = df['without_stopwords'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))

#remove digitsvb
df['lemmatized_text'] = df['lemmatized_text'].str.replace('\d+', '')

#add some more stopwords
newStopWords = ['beer', 'one', 'would', 'get', 'come', 'also', 'oz', 'could']
stop.extend(newStopWords)
df['lemmatized_text'] = df['lemmatized_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df['lemmatized_text'] = df['lemmatized_text'].astype('str')


# add some more stopwords
newStopWords = ['lager', 'porter', 'stout', 'brewery', 'ale', 'apa', 'ipa', 'ipas', 'belgian', 'red', 'brown', 'cream', 'black', 'amber',
               'golden', 'much', 'year', 'worth', 'english', 'german', 'american', 'stuff', 'pilsner', 'old', 'barleywine',
               'ri', 'imperial', 'non', 'dipa', 'iipa', 'dipas', 'pils', 'pilsener', 'sam', 'irish', 'brewpub', 'st',
               'adam', 'ml', 'tripel', 'quad', 'pa', 'ibu', 'ibus', 'series', 'bell', 'belgium', 'boston', 'city', 'coors',
               'dead', 'dfh', 'dog', 'dogfish', 'founder', 'im', 'imo', 'lagunitas', 'left', 'nevada', 'rogue', 'samuel', 'sierra',
               'southern', 'pint']
stop.extend(newStopWords)
df['lemmatized_text'] = df['lemmatized_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

nltk.download('averaged_perceptron_tagger')

df['lemmatized_text'] = df['lemmatized_text'].astype('str')

def nouns_adj(text):
    #Given a string of text, tokenize the text and pull out only the nouns and adjectives.
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

df['adjectives/nouns'] = df['lemmatized_text'].apply(nouns_adj)
df = df.dropna()

#create new dataframe
beer_text = df[['beer_id', 'beer_name', 'style', 'brewery_name', 'Name', 'lemmatized_text', 'adjectives/nouns']]

#drop null values
beer_text = df.dropna(axis=0, subset=['adjectives/nouns'])

#group by beer, think of each beer as a "document"
grouped_beer_text = beer_text.groupby(['beer_id', 'brewery_name', 'beer_name', 'Name', 'style'])['adjectives/nouns'].agg(lambda col: ''.join(col))

#df
beer_text = pd.DataFrame(grouped_beer_text)
beer_text = beer_text.reset_index()

# using lemmatized_text, create the corpus
corpus = beer_text['adjectives/nouns']

# FEATURE EXTRACTION via tfidf vectorizer
tfidf_model = TfidfVectorizer(max_features=500,
                             max_df=0.25,
                             min_df=0.01)

tfidf_matrix = tfidf_model.fit_transform(corpus).todense()
print('The shape of the matrix is:', tfidf_matrix.shape)
tfidf_df = pd.DataFrame(tfidf_matrix)
tfidf_df.columns = sorted(tfidf_model.vocabulary_)

# Calculate the cosine similarity of the matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Construct a reverse mapping of indices and beer names, and drop duplicate names, if any
indices = pd.Series(beer_text.index, index=beer_text['Name']).drop_duplicates()

print(type(indices))
print(type(cosine_sim))

indices.to_pickle('indices')
np.save("cosine_sim.npy", cosine_sim)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
The shape of the matrix is: (1709, 500)
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [ ]:
!pip install streamlit
!pip install pyngrok

In [46]:
%%writefile full.py
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from surprise import Reader, Dataset
from surprise import SVD, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, SVDpp
from surprise.model_selection import GridSearchCV
from collections import defaultdict
from surprise import accuracy

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import random
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity


st.set_page_config(layout="wide") 
st.title('Beer Recommendations')

page_names = ['Existing User', 'New User', 'Help Me Choose']

page = st.radio('Navigation', page_names)

st.write(page)

if page == 'New User':

  from google.colab import drive
  drive.mount('/content/drive')
  data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
  # User 10% of the dataset for testing
  df_item_based = data_read.sample(frac=0.10, random_state=42)



  st.header('Beer Recommendation App - Collaboration Filter')
  st.write('This app was built to help you expand your beer horizons')
  st.write('Data obtained from https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews')

  beer1 = 'Molson Canadian Lager'
  beer2 = 'Guinness Extra Stout Original'
  beer3 = 'Blue Moon Belgian White'
  beer4 = '90 Minute IPA'
  beer5 = 'Nut Brown Ale'
  beer6 = 'Smoked Porter'
  beer7 = 'Gose'



  # User input for new beer
  st.title('Rate these beer on a scale of 0-5. If you have not tried one, rate how interested you are in trying it:')
  with st.form(key='form1'):
    st.write('Molson Canadian Lager')
    rating1 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='1')
    st.write('Guinness Extra Stout Original')
    rating2 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='2')
    st.write('Blue Moon Belgian White')
    rating3 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='3')
    st.write('90 Minute IPA')
    rating4 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='4')
    st.write('Nut Brown Ale')
    rating5 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='5')
    st.write('Smoked Porter')
    rating6 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='6')
    st.write('Gose')
    rating7 = st.slider("Rating", min_value=0.0, max_value=5.0, step=0.25,key='7')
    
    submit = st.form_submit_button(label = 'Press Here for New Beer!')

  text = ['the perfect beer for anyone who loves the summer months hanging with friends on patios and pairing anything barbecue with refreshing brews',
          'the perfect beer for anyone who loves the bitter flavour of black coffee or a strong grapefruit',
          'the perfect beer for anyone who loves coffee-crisp purposely burnt marshmallows and other bold rich flavours',
          'the perfect beer for anyone who loves dried fruit Werther’s Original candy bold flavours and just a pinch of sweet flavours',
          'the perfect beer for anyone who loves scotch campfires and smoked meats smoky brews can be enjoyed all year round',
          'the perfect beer for anyone whos morning routine consists of chugging a glass of orange juice or anyone who never says when when the waiter is adding pepper to your meal',
          'the perfect beers for anyone who drinks green tea in the morning instead of coffee puts cucumber in their water to give it flavour or is semi-addicted to sour penny candies']

  user = [(beer1,rating1),(beer2,rating2),(beer3,rating3),(beer4,rating4),(beer5,rating5),(beer6,rating6),(beer7,rating7)]
  user = pd.DataFrame(user, columns = ['beer_name', 'user_overall_score'])

  user['brewery_name'] =  pd.np.where(user.beer_name.str.contains("Canadian"), 'Molson Coors Canada',
                          pd.np.where(user.beer_name.str.contains("Guinness"), 'Guinness Ltd',
                          pd.np.where(user.beer_name.str.contains("Blue Moon"), 'Coors Brewing Company MolsonCoors',
                          pd.np.where(user.beer_name.str.contains("90 Minute"), 'Dogfish Head Craft Brewery',
                          pd.np.where(user.beer_name.str.contains("Nut Brown Ale"), 'Samuel Smith Old Brewery Tadcaster',
                          pd.np.where(user.beer_name.str.contains("Smoked Porter"), 'Alaskan Brewing Co Alaskan',
                          pd.np.where(user.beer_name.str.contains("Flemish"), 'Brouwerij Van Steenberge NV', 'Westbrook Brewing Co')))))))

  user['beer_id'] = pd.np.where(user.beer_name.str.contains("Canadian"), '1312.0',
                    pd.np.where(user.beer_name.str.contains("Guinness"), '650.0',
                    pd.np.where(user.beer_name.str.contains("Blue Moon"), '1212.0',
                    pd.np.where(user.beer_name.str.contains("90 Minute"), '2093.0',
                    pd.np.where(user.beer_name.str.contains("Nut Brown Ale"), '576.0',
                    pd.np.where(user.beer_name.str.contains("Smoked Porter"), '90.0',
                    pd.np.where(user.beer_name.str.contains("Flemish"), '10482.0', '133043.0')))))))

  user['text'] = pd.np.where(user.beer_name.str.contains("Lager"), text[0],
                pd.np.where(user.beer_name.str.contains("IPA"), text[1],
                pd.np.where(user.beer_name.str.contains("Stout"), text[2],
                pd.np.where(user.beer_name.str.contains("Brown Ale"), text[3],
                pd.np.where(user.beer_name.str.contains("Smoke"), text[4],
                pd.np.where(user.beer_name.str.contains("White"), text[5], text[6]))))))

  new_ratings_df = df_item_based.append(user,ignore_index=True)
  new_ratings_df = new_ratings_df.fillna(0)
  new_ratings_df = new_ratings_df[['username','Name','user_overall_score']]
  new_ratings_df['user_overall_score'] = new_ratings_df['user_overall_score'].astype(float, errors = 'raise')
  ratings = new_ratings_df.pivot_table(index='username', columns='Name', values='user_overall_score')
  ratings.fillna(0,inplace=True)

  # Removing sparsity 
  csr_ratings = csr_matrix(ratings.values)
  # Standardize the mean of each user to 0, and correct for harsh/lenient users
  def standardize(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row

  ratings_std = ratings.apply(standardize)

  # For item-to-item Similarity
  item_similarity = cosine_similarity(ratings_std.T)
  item_similarity_df = pd.DataFrame(item_similarity,index=ratings_std.columns, columns=ratings_std.columns)



  def get_similar_beer(beer_name,user_rating):
    similar_score = item_similarity_df[beer_name]*(user_rating - 2.5)
    similar_score = similar_score.sort_values(ascending = False)
    return similar_score

  # Using user input beer
  user['Name'] = user['brewery_name']+' '+user['beer_name']
  new_user = user[['Name', 'user_overall_score']]
  new_user_list = new_user.to_records(index=False).tolist()

  similar_beer = pd.DataFrame()

  for beer, rating in new_user_list:
    similar_beer = similar_beer.append(get_similar_beer(beer,rating))
    x = similar_beer.sum().sort_values(ascending= False)
    x = pd.DataFrame(x).reset_index()
    x = x.rename(columns={'index': 'Name',0:'Score'})
    y = pd.DataFrame(new_user_list)
    y = y.rename(columns={0: 'Name',1:'Score'})
    x = x[~x['Name'].isin(y['Name'])]
    
  # top_choices = x.head(10)
  # bottom_choices = x.tail(1)
  st.write('Top Beer Choices')
  st.write(x['Name'].head(10))
  st.write('Or Maybe Something Completely Different')
  st.write(x['Name'].tail(1))

elif page == 'Existing User':

  from google.colab import drive
  drive.mount('/content/drive')
  data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
  df = data_read.sample(frac=0.10, random_state=42)
  df = df.rename(columns={'username': 'userID','Name':'itemID','user_overall_score':'rating'})
  df_copy = df.copy()
  df_copy['Name'] = df_copy['brewery_name']+' '+df_copy['beer_name']
  df_copy = df_copy[['userID', 'Name', 'itemID', 'style', 'rating']]

  reader = Reader(rating_scale=(0, 5))
  df = Dataset.load_from_df(df[['userID', 	'itemID', 	'rating' 	]], reader=reader)
  # Creating a train/test set within Surprise
  raw_ratings = df.raw_ratings
  threshold = int(.7 * len(raw_ratings))                                     
  trainset_raw_ratings = raw_ratings[:threshold]                             
  test_raw_ratings = raw_ratings[threshold:]  
  df.raw_ratings = trainset_raw_ratings

  algo_SVD = SVD(n_factors = 4)
  trainset = df.build_full_trainset()
  algo_SVD.fit(trainset)
  print('algorithm: SVD with GridSearchCV')
  # Trainset
  predictions = algo_SVD.test(trainset.build_testset())
  print('Biased accuracy on trainset,', end='   ')
  accuracy.rmse(predictions)
  # Testset
  testset = df.construct_testset(test_raw_ratings)

  predictions = algo_SVD.test(testset)
  print('Unbiased accuracy on testset,', end=' ')
  accuracy.rmse(predictions)


  def get_top_n(predictions, userID, df_copy, n = 10):
      '''Return the top N (default) itemID for a user,.i.e. userID and history for comparison
      Args:
      Returns: 
    
      '''
      #Part I.: Surprise docomuntation
      
      #1. First map the predictions to each user.
      top_n = defaultdict(list)
      for uid, iid, true_r, est, _ in predictions:
          top_n[uid].append((iid, est))

      #2. Then sort the predictions for each user and retrieve the k highest ones.
      for uid, user_ratings in top_n.items():
          user_ratings.sort(key = lambda x: x[1], reverse = True)
          top_n[uid] = user_ratings[: n ]
      
      #Part II.: inspired by: https://beckernick.github.io/matrix-factorization-recommender/
      
      #3. Tells how many beer the user has already rated
      user_data = df_copy[df_copy.userID == (userID)]
      print('User {0} has already rated {1} beer.'.format(userID, user_data.shape[0]))

      
      #4. Data Frame with predictions. 
      preds_df = pd.DataFrame([(id, pair[0],pair[1]) for id, row in top_n.items() for pair in row],
                          columns=["userID" ,"itemID","rat_pred"])
      
      
      #5. Return pred_usr, i.e. top N recommended beer. 
      pred_usr = pd.merge(preds_df[preds_df["userID"] == (userID)],df_copy, on = 'itemID')
      pred_usr = pred_usr.rename(columns={'userID_x': 'userID'})
      pred_usr = pred_usr[['userID', 'itemID', 'style', 'rat_pred']]
              
      #6. Return hist_usr, i.e. top N historically rated beer.
      hist_usr = df_copy[['userID','itemID','style','rating']]
      hist_usr = hist_usr[hist_usr['userID'] == userID]
      
      
      return hist_usr, pred_usr


  st.header('Recommendations for Existing Users')

  # Sample of usernames for testing
  st.write('Sample Usernames')
  st.dataframe(df_copy['userID'].sample(20))

  with st.form(key='form1'):  
    username = st.text_input('Select Username')
    st.form_submit_button(label = 'Press Here for New Beer!')
    hist_SVD, pred_SVD = get_top_n(predictions, df_copy = df_copy, userID = username)


  st.write('Displaying Recommendations for User: ', username)

  pred_SVD = pred_SVD.drop_duplicates('itemID')
  hist_SVD = hist_SVD.drop_duplicates('itemID')
  st.write('Recommendations')
  st.dataframe(pred_SVD)
  st.write('Previous Ratings')
  st.dataframe(hist_SVD)

else:

  import pickle
  indices = pd.read_pickle('indices')
  cosine_sim = np.load("cosine_sim.npy")
  beer_text = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/beer_text.csv')


  st.title('Beer Recommendation App - Content Recommendation')
  st.write('This app was built to help you expand your beer horizons')
  st.write('Data obtained from https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews')


  def content_recommender(title, cosine_sim=cosine_sim, df=beer_text, indices=indices):
      # Obtain the index of the beer that matches the name
      idx = indices[title]
      sim_scores = list(enumerate(cosine_sim[idx]))
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:11]
      beer_indices = [i[0] for i in sim_scores]

      
      return beer_text[['style', 'brewery_name', 'beer_name']].iloc[beer_indices]

  st.write('What are you in the mood for?')

  want = st.selectbox('Select', ['I want a beer for summer, hanging with friends on patios, and barbecueing.',
          'I love the bitter flavour of black coffee or a strong grapefruit.',
          'I love coffee-crisp, purposely burnt marshmallows, and other bold rich flavours.',
          'I love dried fruit, Werther’s Originals, bold flavours, and just a pinch of sweet.',
          'I love scotch, campfires, and smoked meats.',
          "I love a glass of orange juice and I never says 'when' when the waiter is adding pepper to my meal.",
          'I drink green tea instead of coffee, love cucumber in my water, and am semi-addicted to sour candies'])
  if 'barbecue' in want:
    test_beer = 'Molson Coors Canada Molson Canadian Lager'
  elif 'marshmallows' in want:
    test_beer = 'Guinness Ltd Guinness Extra Stout Original'
  elif 'chugging' in want:
    test_beer = 'Coors Brewing Company MolsonCoors Blue Moon Belgian White'
  elif 'grapefruit' in want:
    test_beer = 'Dogfish Head Craft Brewery 90 Minute IPA'
  elif 'Original' in want:
    test_beer = 'Samuel Smith Old Brewery Tadcaster Nut Brown Ale'
  elif 'smoked' in want:
    test_beer = 'Alaskan Brewing Co Alaskan Smoked Porter'
  elif 'cucumber' in want:
    test_beer = 'Westbrook Brewing Co Gose'
  else:
    test_beer = "I'm not a beer person"


  st.write(content_recommender(test_beer))

Overwriting full.py


In [43]:
!ls

cosine_sim.npy	drive  full.py	indices  sample_data


# Insert ngrok authtoken

In [47]:
import os
!ngrok authtoken XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
from pyngrok import ngrok
!streamlit run --server.port 80 full.py&>/dev/null&
publ_url = ngrok.connect(port='80')
publ_url 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


<NgrokTunnel: "http://e7e6-35-186-183-225.ngrok.io" -> "http://localhost:80">

In [45]:
ngrok.kill()